In [1]:
import numpy as np
import igl
import meshplot as mp

from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

In [2]:
# decomposes the mesh into a smoothed version of itself and displacement data 
# that describes surface detail in relation to the smoothed mesh 
def decompose(labels, v, f):
    Lw = igl.cotmatrix(v, f)
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    Minv = sp.diags(1 / M.diagonal())
    
    try:
        A = Lw.dot(Minv.dot(Lw)).todense()
        b = np.zeros(v.shape)
        I = np.identity(A.shape[0])

        for index, point in enumerate(v):
            if labels[index] == 0:
                continue

            b[index] = v[index]
            A[index] = I[index]

        A = sp.csr_matrix(A)
        B = spsolve(A, b)
        
    except:
        A = Lw.dot(Minv.dot(Lw)).tolil()
        b = np.zeros(v.shape)

        for index, point in enumerate(v):
            if labels[index] == 0:
                continue

            b[index] = v[index]
            A[index] = np.zeros(A.shape[0]) 
            A[index, index] = 1.0

        B = spsolve(A.tocsr(), b)
    
    indices = []
    d = np.zeros(v.shape)
    adjacency = igl.adjacency_list(f)
    n = igl.per_vertex_normals(B, f, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)

    for index, point in enumerate(v):
        di = point - B[index]

        T1 = n[index]
        projections = [(vertex - B[index]) - (np.dot(vertex - B[index], T1) * T1) for vertex in B[adjacency[index]]]
        lengths = np.linalg.norm(projections, axis=1)
        i = np.argmax(lengths)
        T2 = projections[i] / lengths[i]
        T3 = np.cross(T1, T2)

        d[index] = np.array([np.dot(di, T1), np.dot(di, T2), np.dot(di, T3)])
        indices.append(i)

    return B, d, indices

# deforms the smoothed mesh
def deform(handle_vertex_positions, v, f):
    Lw = igl.cotmatrix(v, f)
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    Minv = sp.diags(1 / M.diagonal())
    
    try:
        A = Lw.dot(Minv.dot(Lw)).todense()
        b = np.zeros(v.shape)
        I = np.identity(A.shape[0])

        for index, point in enumerate(v):
            if labels[index] == 0:
                continue

            b[index] = handle_vertex_positions[index]
            A[index] = I[index]

        A = sp.csr_matrix(A)
        B = spsolve(A, b)
        
    except:
        A = Lw.dot(Minv.dot(Lw)).tolil()
        b = np.zeros(v.shape)

        for index, point in enumerate(v):
            if labels[index] == 0:
                continue

            b[index] = handle_vertex_positions[index]
            A[index] = np.zeros(A.shape[0]) 
            A[index, index] = 1.0

        B = spsolve(A.tocsr(), b)
    
    return B

# reintroduces surface detail to the deformed mesh
def reconstruct(B, d, indices, v, f):
    n = igl.per_vertex_normals(B, f, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)
    adjacency = igl.adjacency_list(f)
    vprime = np.zeros(v.shape)

    for index, point in enumerate(B):
        T1 = n[index]
        projections = [(vertex - point) - (np.dot(vertex - point, T1) * T1) for vertex in B[adjacency[index]]]
        lengths = np.linalg.norm(projections, axis=1)
        i = indices[index]
        T2 = projections[i] / lengths[i]
        T3 = np.cross(T1, T2)

        vprime[index] = (d[index][0] * T1) + (d[index][1] * T2) + (d[index][2] * T3) + point

    return vprime

In [3]:
v, f = igl.read_triangle_mesh('data/cow.off')
labels = np.load('data/cow.label.npy').astype(int)
handle_vertex_positions = np.load('data/cow.pos.npy')
v -= v.min(axis=0)
v /= v.max()

# the original mesh
p = mp.plot(v, f, c=labels)

smoothed, details, outgoing_edges = decompose(labels, v, f)

# the smoothed mesh, removed surface detail is shown using lines
p = mp.plot(smoothed, f, c=labels)
p.add_lines(v, smoothed)

deformed = deform(handle_vertex_positions, v, f)
result = reconstruct(deformed, details, outgoing_edges, v, f)

# the deformed smoothed mesh
p = mp.plot(deformed, f, c=labels)
p.add_lines(result, deformed)

# the deformed mesh with surface detail reintroduced
p = mp.plot(result, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.30…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5114811…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4900078…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4870488…